In [118]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split, cross_val_score


In [53]:

from main import UserPredictor
model = UserPredictor()
train_users = pd.read_csv("data/train_users.csv")
train_logs = pd.read_csv("data/train_logs.csv")
train_y = pd.read_csv("data/train_y.csv")
# model.fit(train_users, train_logs, train_y)

In [54]:
train_users

,user_id,names,age,past_purchase_amt,badge
0,0,Adriana Mcclure,26,39.344704,gold
1,1,Stacy Gilmore,67,15.840151,silver
2,2,Joanna Walsh,50,1099.420085,bronze
3,3,Eduardo Moore,65,5.880239,bronze
4,4,Angela Freeman,88,1312.296847,bronze
...,...,...,...,...,...
29995,29995,William Lewis,87,108.914205,bronze
29996,29996,James Miranda,49,291.969273,gold
29997,29997,Carlos Davis,46,2.609698,bronze
29998,29998,Makayla Smith,23,164.885894,bronze


In [55]:
train_logs

,user_id,date,url,seconds
0,0,10/27/2021,/keyboard.html,159
1,0,3/15/2021,/blender.html,15
2,0,7/29/2021,/keyboard.html,11
3,0,1/27/2021,/laptop.html,142
4,1,3/1/2021,/keyboard.html,78
...,...,...,...,...
104903,29999,7/5/2021,/keyboard.html,65
104904,29999,3/1/2021,/blender.html,111
104905,29999,7/9/2021,/laptop.html,120
104906,29999,11/13/2021,/tablet.html,24


In [56]:
train_y

,user_id,y
0,0,False
1,1,False
2,2,True
3,3,False
4,4,False
...,...,...
29995,29995,False
29996,29996,True
29997,29997,False
29998,29998,False


In [57]:
test_users = pd.read_csv("data/test1_users.csv")
test_logs = pd.read_csv("data/test1_logs.csv")
test_y = pd.read_csv("data/test1_y.csv")

In [58]:
test_y

,user_id,y
0,30000,True
1,30001,True
2,30002,True
3,30003,True
4,30004,True
...,...,...
29995,59995,True
29996,59996,True
29997,59997,False
29998,59998,True


In [59]:
model = LogisticRegression(fit_intercept = False)
model.fit(train_users[["past_purchase_amt"]],train_y["y"])
model.predict(test_users[["past_purchase_amt"]]).reshape(-1,1)

array([[ True],
       [ True],
       [ True],
       ...,
       [ True],
       [ True],
       [ True]])

In [61]:
train_logs.groupby(by="user_id")["seconds"].sum()

user_id
0        327
1         78
2        432
6        592
8        420
        ... 
29993    657
29994    807
29995    563
29996    532
29999    467
Name: seconds, Length: 26215, dtype: int64

In [102]:
train_users["seconds"] = train_logs.groupby(by="user_id")["seconds"].sum() # create seconds column
train_users['seconds'] = train_users['seconds'].fillna(0) # handle NaN

oh = OneHotEncoder()
badge_df = pd.DataFrame(oh.fit_transform(train_users[["badge"]]).toarray(), columns = oh.get_feature_names_out())
for column_name in oh.get_feature_names_out():
    train_users[column_name] = badge_df[column_name] # create new columns for categorizing badges
    
test_users["seconds"] = test_logs.groupby(by="user_id")["seconds"].sum() # create seconds column
test_users['seconds'] = test_users['seconds'].fillna(0) # handle NaN

oh = OneHotEncoder()
badge_df = pd.DataFrame(oh.fit_transform(test_users[["badge"]]).toarray(), columns = oh.get_feature_names_out())
for column_name in oh.get_feature_names_out():
    test_users[column_name] = badge_df[column_name] # create new columns for categorizing badges

# train_users


In [124]:
xcols = ["past_purchase_amt", "badge", "age", "seconds"]
numericals = ["past_purchase_amt", "age", "seconds"]
categoricals = ["badge"]
custom_trans = make_column_transformer(
    (StandardScaler(), numericals),
    (OneHotEncoder(), categoricals),
)

model = Pipeline([
    ("custom", custom_trans),
    ("lr", LogisticRegression(fit_intercept = False)),
])
model

Pipeline(steps=[('custom',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['past_purchase_amt', 'age',
                                                   'seconds']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['badge'])])),
                ('lr', LogisticRegression(fit_intercept=False))])

In [125]:
model.fit(train_users[xcols], train_y["y"])
model.predict(test_users[xcols])

array([False, False, False, ..., False, False, False])

In [127]:
test_y["y"]
test_y["predictions"] = model.predict(test_users[xcols])
test_y

,user_id,y,predictions
0,30000,True,False
1,30001,True,False
2,30002,True,False
3,30003,True,False
4,30004,True,False
...,...,...,...
29995,59995,True,False
29996,59996,True,False
29997,59997,False,False
29998,59998,True,False


In [128]:
scores = cross_val_score(model, train_users[xcols], train_y["y"])
print(f"AVG: {scores.mean()}, STD: {scores.std()}\n")

AVG: 0.796, STD: 0.004849971363031138

